Pontificia Universidad Católica de Chile <br>
Departamento de Ciencia de la Computación <br>
IIC2433 - Minería de Datos
<br>

<center>
    <h2> Tarea 5 </h2>
    <h1> Ensembles </h1>
    <p>
        Profesor Marcelo Mendoza<br>
        Primer Semestre 2023<br>    
        Fecha de entrega: Miércoles  8 de Junio
    </p>
    <br>
</center>

<br>

---

## Indicaciones

Deberás entregar **SOLO** el archivo .ipynb en el buzón respectivo en canvas. 

**IMPORTANTE**: 
- Se te dará puntaje tanto por código como por la manera en la que respondas las preguntas planteadas. Es decir, si tienes un código perfecto pero este no es explicado o no se responden preguntas asociadas a este, no se tendrá el puntaje completo.
- El notebook debe tener todas las celdas de código ejecutadas. Cualquier notebook que no las tenga no podrá ser corregido.
- El carácter de esta tarea es **INDIVIDUAL**. Cualquier instancia de copia resultará en un 1,1 como nota de curso.

---

In [61]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder

import numpy as np
import pandas as pd

## Parte 1: Carga de datos
En esta tarea debes usar el dataset de Titanic que puedes descargar desde [Kaggle](https://www.kaggle.com/competitions/titanic/data?select=train.csv), solo necesitas el archivo `train.csv`.

Describe el dataset.

In [59]:
df = pd.read_csv("train.csv")

## Parte 2: Preprocesamiento (15 ptos.)
Primero debes separar los datos en conjuntos de training y test (70/30).

Luego, es necesario preprocesar los datos, por lo que debes considerar si es necesario hacer entre otras, las siguientes cosas:
- Remover columnas
- Normalizar variables
- Manejo de valores nulos

En este caso se necesita hacer un paso de preprocesamiento particular. Para manejar los datos categoricos tienes que usar One Hot Encoding.

Recuerda justificar cada paso de preprocesamiento que hagas.

### Preguntas:
- ¿Por que es necesario usar One Hot Encoding?¿Cuál es el problema con usar numeros enteros para codificar las categorias?

## Parte 3: Decision Tree (15 ptos.)
Entrena un Decision Tree y ajusta el parametro que controla la profundidad máxima del árbol, luego reporta las métricas vistas en el curso en los datos de testing.

## Parte 4: Ensembles (30 ptos.)
Entrena los modelos Random Forest y AdaBoost, cada uno usando 50, 100 y 150 Decision Trees. Computa las métricas vistas en el curso y luego graficalas en el eje Y y la cantidad de árboles en el eje X. Compara los resultados con los obtenido en la parte 3.

Luego, ajusta la cantidad de árboles para cada uno de los ensembles, para esto apoyate de un gráfico error vs número de arboles.

### Preguntas:
- Explica en alto nivel, pero de forma completa cómo funcionan cada uno de los algoritmos.